In [1]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score

# Datos

In [2]:
mnist = fetch_openml ('mnist_784', version=1)
mnist_ext = np.append(mnist.data, np.ones((mnist.data.shape[0],1), dtype=float).reshape(-1,1), axis=1)
X = normalize(mnist_ext, axis=1)
Y = mnist.target
XR = X[:700]
YR = np.array([int(mnist.target[i]) for i in range(700)])

# Clases

In [3]:
class neurona(object):
    def __init__(self, f=0, c=0, dim=0):
        self.c = c
        self.f = f
        self.dim = dim
        self.w = normalize(0.5 - np.random.rand(self.dim).reshape(1,-1)) # -1 , 1 ?
        self.output = -10
        self.label = -1

    def predict(self, inputs):
        self.output = inputs @ self.w.T
        return self.output
    
    def fit(self, inputs, alfa=1):
        self.w = normalize(self.w + (alfa*inputs))
        
    def labeling(self, inputs, target):
        Y = inputs @ self.w.T
        self.label = target[np.argmax(Y)]
      

In [4]:
class som():
    
    def __init__(self, filas=1, columnas=1, dim=1):
        self.filas = filas
        self.columnas = columnas
        self.dim = dim
        self.neuron = [[neurona(f=i, c=j, dim=self.dim)for j in range(self.columnas)]for i in range(self.filas)]
        
    def fit(self, inputs, max_epochs=1, init_radious=0, init_alfa=1):
        self.radious = init_radious
        self.alfa = init_alfa
        t = 0
        P = inputs.shape[0]
        for epoch in range(max_epochs):
            for x in inputs:
                self.alfa = init_alfa/(1.0 + float(t/P))
                f_gana, c_gana, y_gana = -1, -1, float('-inf')
                for f in range(self.filas):
                    for c in range (self.columnas):
                        y_predict = self.neuron[f][c].predict(x)
                        if y_predict > y_gana:
                            f_gana = f
                            c_gana = c
                            y_gana = y_predict
                            
                # Conjunto de vecinos par un radious
                for f in range(f_gana - self.radious, f_gana + self.radious+1):
                    if f < 0:
                        row = self.filas + f
                    else:
                        if f > self.filas-1:
                            row = f % self.filas
                        else:
                            row = f
                    for c in range(c_gana - self.radious, c_gana + self.radious+1):
                        if c < 0:
                            column = self.columnas + c
                        else:
                            if c > self.columnas-1:
                                column = c % self.columnas
                            else:
                                column = c
                        self.neuron[row][column].fit(x.reshape(1,-1),alfa=self.alfa)
                t += 1
                if (t%(P/10)) == 0:
                    print(t, self.radious, ' ', end='')
            if self.radious > 0:
                self.radious -= 1
    
    def neuron_labeling(self, inputs, target):
        for f in range(self.filas):
            for c in range(self.columnas):
                self.neuron[f][c].labeling(inputs, target)
                
    def output_predict(self, inputs):
        output = np.zeros((inputs.shape[0], self.filas, self.columnas), dtype=float)
        for i,x in enumerate(inputs):
            for f in range(self.filas):
                for c in range(self.columnas):
                    output[i,f,c] = self.neuron[f][c].predict(x)
        return output
    
    def label_predict(self, x):
        f_gana, c_gana, y_gana = -1,-1,float('-inf')
        for f in range(self.filas):
            for c in range(self.columnas):
                y_predict = self.neuron[f][c].predict(x)
                if y_predict > y_gana:
                    f_gana = f
                    c_gana = c
                    y_gana = y_predict
        return self.neuron[f_gana][c_gana].label
                            

# Ejemplo corto 

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(XR, YR, test_size=100, stratify=YR)
mapa_autoorganizado = som(filas=15, columnas=9,dim=X_train.shape[1])
mapa_autoorganizado.fit(inputs=X_train, max_epochs=20, init_radious=4, init_alfa=10)
mapa_autoorganizado.neuron_labeling(inputs=X_train,target=Y_train)
etiquetas_neuronas_predict= [mapa_autoorganizado.label_predict(x) for x in X_test]
accuracy = accuracy_score(Y_test, etiquetas_neuronas_predict)
accuracy

60 4  120 4  180 4  240 4  300 4  360 4  420 4  480 4  540 4  600 4  660 3  720 3  780 3  840 3  900 3  960 3  1020 3  1080 3  1140 3  1200 3  1260 2  1320 2  1380 2  1440 2  1500 2  1560 2  1620 2  1680 2  1740 2  1800 2  1860 1  1920 1  1980 1  2040 1  2100 1  2160 1  2220 1  2280 1  2340 1  2400 1  2460 0  2520 0  2580 0  2640 0  2700 0  2760 0  2820 0  2880 0  2940 0  3000 0  3060 0  3120 0  3180 0  3240 0  3300 0  3360 0  3420 0  3480 0  3540 0  3600 0  3660 0  3720 0  3780 0  3840 0  3900 0  3960 0  4020 0  4080 0  4140 0  4200 0  4260 0  4320 0  4380 0  4440 0  4500 0  4560 0  4620 0  4680 0  4740 0  4800 0  4860 0  4920 0  4980 0  5040 0  5100 0  5160 0  5220 0  5280 0  5340 0  5400 0  5460 0  5520 0  5580 0  5640 0  5700 0  5760 0  5820 0  5880 0  5940 0  6000 0  6060 0  6120 0  6180 0  6240 0  6300 0  6360 0  6420 0  6480 0  6540 0  6600 0  6660 0  6720 0  6780 0  6840 0  6900 0  6960 0  7020 0  7080 0  7140 0  7200 0  7260 0  7320 0  7380 0  7440 0  7500 0  7560 0  7620 0  7

0.83

# Datos completos

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=10000, stratify=Y)
mapa_autoorganizado = som(filas=15, columnas=9,dim=X_train.shape[1])
mapa_autoorganizado.fit(inputs=X_train, max_epochs=20, init_radious=4, init_alfa=10)
mapa_autoorganizado.neuron_labeling(inputs=X_train,target=Y_train)
etiquetas_neuronas_predict= [mapa_autoorganizado.label_predict(x) for x in X_test]
accuracy = accuracy_score(Y_test, etiquetas_neuronas_predict)
accuracy

6000 4  12000 4  18000 4  24000 4  30000 4  36000 4  42000 4  48000 4  54000 4  60000 4  66000 3  72000 3  78000 3  84000 3  90000 3  96000 3  102000 3  108000 3  114000 3  120000 3  126000 2  132000 2  138000 2  144000 2  150000 2  156000 2  162000 2  168000 2  174000 2  180000 2  186000 1  192000 1  198000 1  204000 1  210000 1  216000 1  222000 1  228000 1  234000 1  240000 1  246000 0  252000 0  258000 0  264000 0  270000 0  276000 0  282000 0  288000 0  294000 0  300000 0  306000 0  312000 0  318000 0  324000 0  330000 0  336000 0  342000 0  348000 0  354000 0  360000 0  366000 0  372000 0  378000 0  384000 0  390000 0  396000 0  402000 0  408000 0  414000 0  420000 0  426000 0  432000 0  438000 0  444000 0  450000 0  456000 0  462000 0  468000 0  474000 0  480000 0  486000 0  492000 0  498000 0  504000 0  510000 0  516000 0  522000 0  528000 0  534000 0  540000 0  546000 0  552000 0  558000 0  564000 0  570000 0  576000 0  582000 0  588000 0  594000 0  600000 0  606000 0  612000 

0.8521